In [102]:
import config as cfg
import os
import pandas as pd
import networkx as nx
import geopandas as gpd
from shapely.geometry import Polygon
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import plotly.graph_objects as go

# Load data
all_viajes = pd.read_csv('/Users/caro/Desktop/thesis_project/mobility_data/VIAJES/all_viajes_week_0222.csv')
gdf = gpd.read_file(cfg.ZONIFICACION_DATA / 'distritos/madrid_gdf.geojson')  # Load your GeoJSON file into a GeoDataFrame
gdf = gdf.to_crs(epsg=3042)

# Filter data
filtered_df = all_viajes.loc[(all_viajes['actividad_origen'] == 'casa')]

In [104]:
filtered_df.head()

,fecha,periodo,origen,destino,distancia,actividad_origen,actividad_destino,estudio_origen_posible,estudio_destino_posible,residencia,renta,edad,sexo,viajes,viajes_km
0,20220207,0,2807901,2807901,0.5-2,casa,frecuente,no,no,28,>15,0-25,hombre,20.700,14.144
1,20220207,0,2807901,2807901,0.5-2,casa,frecuente,no,no,28,>15,0-25,mujer,25.476,15.796
2,20220207,0,2807901,2807901,0.5-2,casa,frecuente,no,no,28,>15,25-45,hombre,102.263,69.730
3,20220207,0,2807901,2807901,0.5-2,casa,frecuente,no,no,28,>15,25-45,mujer,86.178,63.775
4,20220207,0,2807901,2807901,0.5-2,casa,frecuente,no,no,28,>15,45-65,hombre,43.469,31.687


In [112]:
# DEFINING GRAPH --------------------------------------------------------------------------------------------------------- 

# Function to generate node positions based on GeoDataFrame
def get_positions(gdf):
    return {
        int(row['ID']): (row['geometry'].centroid.x, row['geometry'].centroid.y)
        for idx, row in gdf.iterrows()
    }

# Define the graph based on DataFrame
def define_graph(df):
    G = nx.DiGraph()

    # Group by origin and destination, and aggregate trip count and renta (taking the first renta value)
    trip_counts = df.groupby(['origen', 'destino', 'renta']).size().reset_index(name='trip_count')
    
    # Add edges to the graph
    for idx, row in trip_counts.iterrows():
        G.add_edge(row['origen'], row['destino'], weight=row['trip_count'], renta=row['renta'])  # Save 'renta' as edge attribute
    return G, trip_counts

# PLOTTING ---------------------------------------------------------------------------------------------------------

# Set edge attributes like color and width for visualization
def set_art(G, weight_scale=6000):
    edge_colors = []
    edge_widths = []
    
    # Iterate over the edges and set colors based on 'renta' attribute
    for u, v, data in G.edges(data=True):
        # Choose color based on 'renta' value (numerical or categorical)
        if data['renta'] == '>15':
            edge_colors.append('blue')
        elif data['renta'] == '10-15':
            edge_colors.append('red')
        else:
            edge_colors.append('green')

        # Adjust edge width based on the weight (i.e., trip count)
        edge_widths.append(max(0.5, data['weight'] / weight_scale))
    
    return edge_colors, edge_widths

def plotly_graph(G, positions, edge_colors, edge_widths, node_size=20):
    # Extract node positions
    node_x = [positions[node][0] for node in G.nodes()]
    node_y = [positions[node][1] for node in G.nodes()]

    # Create node scatter trace
    node_trace = go.Scatter(
        x=node_x, y=node_y,
        mode='markers',
        marker=dict(size=node_size, color='white', line=dict(width=2, color='#888')),
        text=[f'Node {node}' for node in G.nodes()],
        hoverinfo='text'
    )

    # Prepare edges in batches based on edge color (blue, red, green)
    edge_traces = []
    for edge_color in ['blue', 'green', 'red']:
        edge_x, edge_y = [], []
        for i, (u, v, data) in enumerate(G.edges(data=True)):
            if edge_colors[i] == edge_color:
                x0, y0 = positions[u]
                x1, y1 = positions[v]
                edge_x.extend([x0, x1, None])
                edge_y.extend([y0, y1, None])

        # Create edge trace for each color
        edge_traces.append(
            go.Scatter(
                x=edge_x, y=edge_y,
                line=dict(width=edge_widths[i], color=edge_color),
                hoverinfo='none',
                mode='lines'
            )
        )

    # Create a layout for the graph
    layout = go.Layout(
        showlegend=True,
        hovermode='closest',
        margin=dict(b=0, l=0, r=0, t=0),
        xaxis=dict(showgrid=False, zeroline=False, showticklabels=False),
        yaxis=dict(showgrid=False, zeroline=False, showticklabels=False)
    )

    # Combine edge and node traces
    fig = go.Figure(data=edge_traces + [node_trace], layout=layout)
    fig.show()

# ANALYSIS ---------------------------------------------------------------------------------------------------------

def check_in_weights(G):
    in_weights = {}
    for node in G.nodes():
        total_in_weight = sum(data['weight'] for u, v, data in G.in_edges(node, data=True))
        in_weights[node] = total_in_weight
        print(f"Node {node} Total In-weight: {total_in_weight}")
    return in_weights

def check_out_weights(G):
    out_weights = {}
    for node in G.nodes():
        total_out_weight = sum(data['weight'] for u, v, data in G.out_edges(node, data=True))
        out_weights[node] = total_out_weight
        print(f"Node {node} Total Out-weight: {total_out_weight}")
    return out_weights

def print_node_degrees(G):
    print("Node Degrees (In-degree, Out-degree, Total degree):")
    for node in G.nodes():
        in_degree = G.in_degree(node)
        out_degree = G.out_degree(node)
        total_degree = G.degree(node)  # Total degree (in + out)
        print(f"Node {node}: In-degree = {in_degree}, Out-degree = {out_degree}, Total degree = {total_degree}")



In [113]:
G, trip_counts = define_graph(filtered_df)
positions = get_positions(gdf)
edge_colors, edge_widths = set_art(G)
plotly_graph(G, positions, edge_colors, edge_widths)

In [ ]:
len(G.edges)

In [107]:
G_unfiltered, trip_counts_unfiltered = define_graph(all_viajes)
edge_colors_unfiltered, edge_widths_unfiltered = set_art(G_unfiltered)
plotly_graph(G_unfiltered, positions, edge_colors_unfiltered, edge_widths_unfiltered)

In [118]:
nx.degree(G, weight='weight')

DiDegreeView({2807901: 88419, 2807902: 73982, 2807903: 72850, 2807904: 81479, 2807905: 79594, 2807906: 71831, 2807907: 80058, 2807908: 86776, 2807909: 80795, 2807910: 73204, 2807911: 78778, 2807912: 67114, 2807913: 87962, 2807914: 61709, 2807915: 87860, 2807916: 75808, 2807917: 60475, 2807918: 67076, 2807919: 59495, 2807920: 79106, 2807921: 49023})

In [121]:
nx.betweenness_centrality(G, weight='weight')

{2807901: 0.0,
 2807902: 0.002631578947368421,
 2807903: 0.0,
 2807904: 0.0,
 2807905: 0.0,
 2807906: 0.0,
 2807907: 0.0,
 2807908: 0.0,
 2807909: 0.002631578947368421,
 2807910: 0.007894736842105263,
 2807911: 0.0,
 2807912: 0.03684210526315789,
 2807913: 0.0,
 2807914: 0.05131578947368421,
 2807915: 0.0,
 2807916: 0.0,
 2807917: 0.042105263157894736,
 2807918: 0.0,
 2807919: 0.04868421052631579,
 2807920: 0.0,
 2807921: 0.22105263157894736}

In [96]:
print_node_degrees(G)

Node Degrees (In-degree, Out-degree, Total degree):
Node 2807901: In-degree = 21, Out-degree = 21, Total degree = 42
Node 2807902: In-degree = 21, Out-degree = 21, Total degree = 42
Node 2807903: In-degree = 21, Out-degree = 21, Total degree = 42
Node 2807904: In-degree = 21, Out-degree = 21, Total degree = 42
Node 2807905: In-degree = 21, Out-degree = 21, Total degree = 42
Node 2807906: In-degree = 21, Out-degree = 21, Total degree = 42
Node 2807907: In-degree = 21, Out-degree = 21, Total degree = 42
Node 2807908: In-degree = 21, Out-degree = 21, Total degree = 42
Node 2807909: In-degree = 21, Out-degree = 21, Total degree = 42
Node 2807910: In-degree = 21, Out-degree = 21, Total degree = 42
Node 2807911: In-degree = 21, Out-degree = 21, Total degree = 42
Node 2807912: In-degree = 21, Out-degree = 21, Total degree = 42
Node 2807913: In-degree = 21, Out-degree = 21, Total degree = 42
Node 2807914: In-degree = 21, Out-degree = 21, Total degree = 42
Node 2807915: In-degree = 21, Out-degr

In [97]:
# Check total in-weights for each node
in_weights = check_in_weights(G)


Node 2807901 Total In-weight: 50686
Node 2807902 Total In-weight: 36671
Node 2807903 Total In-weight: 39836
Node 2807904 Total In-weight: 48533
Node 2807905 Total In-weight: 46176
Node 2807906 Total In-weight: 37448
Node 2807907 Total In-weight: 47954
Node 2807908 Total In-weight: 46012
Node 2807909 Total In-weight: 44784
Node 2807910 Total In-weight: 29161
Node 2807911 Total In-weight: 33909
Node 2807912 Total In-weight: 30019
Node 2807913 Total In-weight: 36509
Node 2807914 Total In-weight: 26717
Node 2807915 Total In-weight: 41327
Node 2807916 Total In-weight: 38545
Node 2807917 Total In-weight: 25250
Node 2807918 Total In-weight: 30283
Node 2807919 Total In-weight: 27292
Node 2807920 Total In-weight: 36997
Node 2807921 Total In-weight: 27588


In [98]:
# Check total out-weights for each node
out_weights = check_out_weights(G)


Node 2807901 Total Out-weight: 37733
Node 2807902 Total Out-weight: 37311
Node 2807903 Total Out-weight: 33014
Node 2807904 Total Out-weight: 32946
Node 2807905 Total Out-weight: 33418
Node 2807906 Total Out-weight: 34383
Node 2807907 Total Out-weight: 32104
Node 2807908 Total Out-weight: 40764
Node 2807909 Total Out-weight: 36011
Node 2807910 Total Out-weight: 44043
Node 2807911 Total Out-weight: 44869
Node 2807912 Total Out-weight: 37095
Node 2807913 Total Out-weight: 51453
Node 2807914 Total Out-weight: 34992
Node 2807915 Total Out-weight: 46533
Node 2807916 Total Out-weight: 37263
Node 2807917 Total Out-weight: 35225
Node 2807918 Total Out-weight: 36793
Node 2807919 Total Out-weight: 32203
Node 2807920 Total Out-weight: 42109
Node 2807921 Total Out-weight: 21435


In [99]:
nx.community.louvain_communities(G)

[{2807904, 2807905, 2807906, 2807907, 2807908, 2807909},
 {2807903, 2807913, 2807914, 2807918, 2807919},
 {2807915, 2807916, 2807920, 2807921},
 {2807901, 2807902, 2807910, 2807911, 2807912, 2807917}]